<center style = "font-family: 'Lucida Console', 'Courier New', monospace;">
    <img src = "https://www.bargainmarket.com.au/images/banner-pets-dog-cat-boarding.png" width=600 height = 400>
    <h1 style="background: url('https://thumbs.dreamstime.com/b/abstract-light-brown-blur-lights-background-design-162886080.jpg')">PetFinder.my - Pawpularity Contest 🐶🐱</h1>
</center>

<br>
<h2 style = "font-family: Consolas">What is PetFinder.my❓</h2>
<p style = "font-family : Lucida Sans Typewriter">
<a href = "https://petfinder.my/">PetFinder.my</a> is Malaysia’s leading animal welfare platform, featuring over 180,000 animals with 54,000 happily adopted. PetFinder collaborates closely with animal lovers, media, corporations, and global organizations to improve animal welfare.
</p>

<br>
<h2 style = "font-family: Consolas">What should we do📝</h2>
<p style = "font-family : Lucida Sans Typewriter">
In this competition, you’ll analyze raw images and metadata to predict the “Pawpularity” of pet photos. You'll train and test your model on PetFinder.my's thousands of pet profiles.</p>

<br>
<h2 style = "font-family: Consolas">Data Information💽</h2>
<p style = "font-family : Lucida Sans Typewriter">Check <a href = "https://www.kaggle.com/c/petfinder-pawpularity-score/data">competition page</a> for details</p>

<h2 style = "font-family : Comic Sans MS">Let's dive in ⬇️</h2>

<center style = "background: rgb(238,174,202);
background: radial-gradient(circle, rgba(238,174,202,1) 0%, rgba(81,154,204,1) 0%);"><h1 style = "font-family:'Bodoni MT'">Libraries📚</h1></center>

In [ ]:
##--------------------------
# Import necessary libraries
##--------------------------
import os

import pandas as pd
from pandas_profiling import ProfileReport

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

import numpy as np

from PIL import Image

import warnings
warnings.filterwarnings('ignore')

<center style = "background: rgb(238,174,202);
background: radial-gradient(circle, rgba(238,174,202,1) 0%, rgba(81,154,204,1) 0%);"><h1 style = "font-family:'Bodoni MT'">Initializations🎬</h1></center>

In [ ]:
TRAIN_PATH = "../input/petfinder-pawpularity-score/train"
TEST_PATH = "../input/petfinder-pawpularity-score/test"

train_df = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
test_df = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")

TRAIN_NS = os.listdir(TRAIN_PATH) 
TEST_NS = os.listdir(TEST_PATH)
TRAIN_PS = [os.path.join(TRAIN_PATH,f) for f in TRAIN_NS]
TEST_PS = [os.path.join(TEST_PATH,f) for f in TEST_NS]

HEIGHT, WIDTH = 224, 224
CHANNELS = 3

<center style = "background: rgb(238,174,202);
background: radial-gradient(circle, rgba(238,174,202,1) 0%, rgba(81,154,204,1) 0%);"><h1 style = "font-family:'Bodoni MT'">Data Visualization📊</h1></center>

In [ ]:
##------------------------------
# Display Random Training Images
##------------------------------

rand_ls = np.random.randint(9000, size=(9))
fig,ax = plt.subplots(nrows=3,ncols=3,figsize=(18,10))
for count,i in enumerate(rand_ls):
    r,c = count//3, count%3
    ax[r,c].imshow(np.asarray(Image.open(TRAIN_PS[i])))
    ax[r,c].axis("off")
    label = train_df[train_df["Id"] == TRAIN_NS[i].split(".")[0]]["Pawpularity"].values[0]
    ax[r,c].set_title(f"Pawpularity Score : {label}",fontsize=15, fontfamily="monospace", fontweight="bold")

In [ ]:
##--------------------------------
# Height, Width of Training Images
##--------------------------------
train_h, train_w = [], []

for path in TRAIN_PS:
    im = Image.open(path)
    w,h = im.size
    train_h.append(h)
    train_w.append(w)
    
plt.figure(figsize=(10,10))
sns.jointplot(x=train_w, y=train_h,kind="kde")
plt.xlabel("Width of Image")
plt.ylabel("Height of Image")

In [ ]:
##---------------------------
# Metadata of Training images
# Pandas Profiling
##---------------------------
train_df["Width"] = train_w
train_df["Height"] = train_h

train_report = ProfileReport(train_df,title="Metadata of Training images")
#train_report.to_file("./train_metadata.html")
train_report

<center style = "background: rgb(238,174,202);
background: radial-gradient(circle, rgba(238,174,202,1) 0%, rgba(81,154,204,1) 0%);"><h1 style = "font-family:'Bodoni MT'">Data Preprocessing🧰</h1></center>

In [ ]:
tot_cols = list(train_df.columns)
feature_cols = tot_cols[1:13]

train_ds = np.array(train_df[feature_cols])
test_ds = np.array(test_df[feature_cols])
#train_y = np.array(train_df["Pawpularity"])

bin_ranges = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
bin_names = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

train_df["label"] = pd.cut(np.array(train_df["Pawpularity"]),bins=bin_ranges,labels=bin_names)

paw_values = []
for i in bin_names:
    val = list(train_df[train_df["label"] == i]["Pawpularity"])
    paw_values.append(sum(val)/len(val))

train_y = np.array(train_df["label"])

<center style = "background: rgb(238,174,202);
background: radial-gradient(circle, rgba(238,174,202,1) 0%, rgba(81,154,204,1) 0%);"><h1 style = "font-family:'Bodoni MT'">Training💡</h1></center>

In [ ]:
%%capture
!pip uninstall -y scikit-learn 
!pip install -q '../input/sklearn24/scikit_learn-0.24.0-cp37-cp37m-manylinux2010_x86_64.whl'

!pip uninstall -y xgboost
!pip install -q '../input/xgboost13123py3nonemanylinux/xgboost-1.3.2/xgboost-1.3.3-py3-none-manylinux2010_x86_64.whl'

!pip install flaml --no-index --find-links=file:///kaggle/input/flamlmain/FLAML-main/

In [ ]:
import flaml 
from flaml import AutoML

automl = AutoML()
settings = {
    "time_budget": 360,  
    "metric": 'accuracy',                  
    "task": 'classification',  
    "log_file_name": 'pawpularity.log',  
    "seed": 2021,
    "estimator_list" : ["lgbm", "xgboost", "catboost"]
}
automl.fit(X_train=train_ds, y_train=train_y, **settings)

<center style = "background: rgb(238,174,202);
background: radial-gradient(circle, rgba(238,174,202,1) 0%, rgba(81,154,204,1) 0%);"><h1 style = "font-family:'Bodoni MT'">Inference✔️</h1></center>

In [ ]:
test_pred = automl.predict(test_ds)
test_pred = [paw_values[i] for i in test_pred]
test_ids = list(test_df["Id"])

pred_df = pd.DataFrame.from_dict({
    "Id" : test_ids,
    "Pawpularity" : test_pred
})
pred_df.to_csv("./submission.csv",index=False)
os.remove("./pawpularity.log")
pred_df.head()